# Train Configuration Files

Cada treinamento vai ter um identificador hash que vai permear todo o processo.

In [ ]:
!pip install ..

In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import src
from src.functions.AuxiliarFunctions import *

In [3]:
# Configurações gerais
hash_id = hash('treinamento1')
label = 'ShipsEar NN Classification' # etiqueta do treinamento
wav_files_path = '../data/shipsEar_AUDIOS' # caminho para os arquivos wav
processed_file_folder =  '../data' # caminho para o arquivo único com todos os dados crus - processed_data.csv
processed_file_path =  os.path.join('../data','%s_processed_data.csv'%(hash_id)) # caminho para o arquivo único com todos os dados crus - processed_data.csv
wav_files_info = os.path.join(processed_file_folder,'wav_file_informations.csv')

# Configurações da Validação Cruzada
cv_alg = 'StratifiedKFolds'
cv_folds = [5]
cv_path = os.path.join(processed_file_folder,'indexes')

# Configurações de Pré-Processamento
preproc_alg = 'Lofar'
preproc_n_fft_points = 1024
preproc_overlap = 0
preproc_spectrum_bins_left = 400

# Configurações de Pipeline
pipeline_path = os.path.join(processed_file_folder,'pipelines')
scaler_alg = 'StandardScaler'

# Arquivos de Treinamento
train_data_path = os.path.join(processed_file_folder,'%s_train_data.csv'%(hash_id)) # caminho para o arquivo único com todos os dados para treinamento - train_data_file.csv
train_trgt_path = os.path.join(processed_file_folder,'%s_trgt_data.csv'%(hash_id)) # caminho para o arquivo único com todos os alvos para treinamento - train_trgt_file.csv
target_label_file = os.path.join(processed_file_folder,'models/train_label_file.csv')
target_id_file = os.path.join(processed_file_folder,'models/train_id_file.csv')
model_path = os.path.join(processed_file_folder,'models') # caminho para onde os modelos serão enviados
model_inits = 1 # numero de inicializações que serão feitas durante o treinamento
model_neurons = '../data/models/%s_hidden_neurons.pkl'%(hash_id) # arquivo com todas as possibilidade de quantidade de neuronios na camada escondida
model_status = '../data/models/%s_model_status.pkl'%(hash_id) # arquivo com todas as possibilidade de quantidade de neuronios na camada escondida

In [4]:
dict_buffer = { 
    'hash_id':[hash_id],
    'label':[label],
    'wav_files_path':[wav_files_path],
    'processed_file_folder':[processed_file_folder],
    'processed_file_path':[processed_file_path],
    'wav_files_info':[wav_files_info],
    'cv_alg':[cv_alg],
    'cv_folds':cv_folds,
    'cv_path':cv_path,
    'preproc_alg':[preproc_alg],
    'preproc_n_fft_points':[preproc_n_fft_points],
    'preproc_overlap':[preproc_overlap],
    'preproc_spectrum_bins_left':[preproc_spectrum_bins_left],
    'pipeline_path':[pipeline_path],
    'scaler_alg':[scaler_alg],
    'train_data_path':[train_data_path],
    'train_trgt_path':[train_trgt_path],
    'target_label_file':[target_label_file],
    'target_id_file':[target_id_file],
    'model_path':[model_path],
    'model_inits':model_inits,
    'model_neurons':[model_neurons],
    'model_status':[model_status]
}
df_buffer = pd.DataFrame(data=dict_buffer)

In [5]:
df_buffer.head()

,hash_id,label,wav_files_path,processed_file_folder,processed_file_path,wav_files_info,cv_alg,cv_folds,cv_path,preproc_alg,...,pipeline_path,scaler_alg,train_data_path,train_trgt_path,target_label_file,target_id_file,model_path,model_inits,model_neurons,model_status
0,8427254357588219694,ShipsEar NN Classification,../data/shipsEar_AUDIOS,../data,../data/8427254357588219694_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,Lofar,...,../data/pipelines,StandardScaler,../data/8427254357588219694_train_data.csv,../data/8427254357588219694_trgt_data.csv,../data/models/train_label_file.csv,../data/models/train_id_file.csv,../data/models,1,../data/models/8427254357588219694_hidden_neur...,../data/models/8427254357588219694_model_statu...


In [6]:
config_file_path = '../data/config.csv'
if os.path.exists(config_file_path) == False:
    # arquivo não existe
    df_buffer.to_csv(config_file_path, index=False)
else:
    # arquivo existe
    df_old = pd.read_csv(config_file_path)
    if (hash_id in list(df_old['hash_id'])):
        # o hash_id já está salvo
        print('o hash_id já está salvo')
    else:
        df_old = pd.concat([df_old,df_buffer],axis=0, ignore_index=True)
        df_old.to_csv(config_file_path)

In [7]:
train_id = 0
df_config = pd.read_csv(config_file_path)

In [8]:
hidden_neurons = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#hidden_neurons = [1]
if os.path.exists(df_config['model_neurons'][train_id]) == False:
    write_list_of_hidden_neurons(df_config['model_neurons'][train_id],hidden_neurons)
else:
    print('O arquivo %s já existe'%(df_config['model_neurons'][train_id]))

O arquivo ../data/models/3359727688101666398_hidden_neurons.pkl já existe


In [9]:
if os.path.exists(df_config['model_status'][train_id]) == False:
    hidden_neurons = get_list_of_hidden_neurons(df_config['model_neurons'][train_id])
    model_status = np.zeros([df_config['cv_folds'][train_id],len(hidden_neurons),df_config['model_inits'][train_id]],dtype=bool)
    with open(df_config['model_status'][train_id],'wb') as file_handler:
        pickle.dump([model_status],file_handler)
else:
    print('O arquivo %s já existe'%(df_config['model_status'][train_id]))

O arquivo ../data/models/3359727688101666398_model_status.pkl já existe
